In [4]:
import pandas as pd
import gurobipy as gp
from gurobipy import GRB
import numpy as np

# Load the data
data = pd.read_csv('sp500_data.csv')

In [6]:
# Create a new model
model = gp.Model("S&P500_Portfolio_Optimization")

# Define decision variables
x = {}
for i in range(len(data)):
    x[i] = model.addVar(lb=0, ub=600000, name=f"x_{i}")

# Define sets for each sector
telecom_indices = [i for i, sector in enumerate(data['GICS Sector']) if sector == 'Telecommunications']
it_indices = [i for i, sector in enumerate(data['GICS Sector']) if sector == 'Information Technology']
energy_indices = [i for i, sector in enumerate(data['GICS Sector']) if sector == 'Energy']
con_disc_indices = [i for i, sector in enumerate(data['GICS Sector']) if sector == 'Consumer Discretionary']
con_staples_indices = [i for i, sector in enumerate(data['GICS Sector']) if sector == 'Consumer Staples']

# Define set for New York companies
ny_indices = [i for i, location in enumerate(data['Location of Headquarters']) if 'New York, NY' in str(location)]

# Objective function: Maximize expected return
model.setObjective(gp.quicksum(data.iloc[i]['PercentReturn']/100 * x[i] for i in range(len(data))), GRB.MAXIMIZE)

# Constraint: Total investment = $10 million
model.addConstr(gp.quicksum(x[i] for i in range(len(data))) == 10000000, "Budget")

# Constraint: No more than $500,000 in Telecommunications sector
model.addConstr(gp.quicksum(x[i] for i in telecom_indices) <= 500000, "Telecom_Restriction")

# Constraint: IT investment ≥ 75% of Telecommunications investment
model.addConstr(gp.quicksum(x[i] for i in it_indices) >= 0.75 * gp.quicksum(x[i] for i in telecom_indices), "IT_Telecom_Restriction")

# Constraint: Absolute difference between Consumer Discretionary and Consumer Staples ≤ $200,000
model.addConstr(gp.quicksum(x[i] for i in con_disc_indices) - gp.quicksum(x[i] for i in con_staples_indices) <= 200000, "Disc_Staples_1")
model.addConstr(gp.quicksum(x[i] for i in con_staples_indices) - gp.quicksum(x[i] for i in con_disc_indices) <= 200000, "Disc_Staples_2")

# Constraint: At least $1 million in Energy sector
model.addConstr(gp.quicksum(x[i] for i in energy_indices) >= 1000000, "Energy_Sector")

# Constraint: At least $300,000 in companies headquartered in New York
model.addConstr(gp.quicksum(x[i] for i in ny_indices) >= 300000, "NYC_Investment")

# Solve the model
model.optimize()

# Display the results
if model.status == GRB.OPTIMAL:
    print(f"Optimal objective value: ${model.objVal:.2f}")
    
    # Investment by sector
    sector_investment = {}
    for sector in data['GICS Sector'].unique():
        indices = [i for i, s in enumerate(data['GICS Sector']) if s == sector]
        investment = sum(x[i].x for i in indices)
        sector_investment[sector] = investment
        print(f"Investment in {sector}: ${investment:.2f}")
    
    # Investment in NYC
    nyc_investment = sum(x[i].x for i in ny_indices)
    print(f"Investment in NYC companies: ${nyc_investment:.2f}")
    
    # Show top investments
    investments = [(i, x[i].x) for i in range(len(data)) if x[i].x > 0]
    investments.sort(key=lambda x: x[1], reverse=True)
    
    print("\nTop investments:")
    for i, amount in investments[:10]:  # Show top 10 investments
        print(f"{data.iloc[i]['Company']}: ${amount:.2f}")
else:
    print("No optimal solution found")


Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 24.3.0 24D70)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 7 rows, 67 columns and 117 nonzeros
Model fingerprint: 0x5f89d287
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-03, 1e-01]
  Bounds range     [6e+05, 6e+05]
  RHS range        [2e+05, 1e+07]
Presolve removed 1 rows and 0 columns
Presolve time: 0.02s

Solved in 0 iterations and 0.03 seconds (0.00 work units)
Infeasible model
No optimal solution found


In [7]:
import pandas as pd
import gurobipy as gp
from gurobipy import GRB
import numpy as np

# Load the data
df = pd.read_csv('sp500_data.csv')

# Create a new model
model = gp.Model("S&P500_Portfolio_Optimization")

# Define decision variables (how much to invest in each stock)
x = {}
for i in range(len(df)):
    x[i] = model.addVar(lb=0, ub=600000, name=f"invest_{i}")

# Define sets for each sector
telecom_indices = [i for i, sector in enumerate(df['GICS Sector']) if sector == 'Telecommunications']
it_indices = [i for i, sector in enumerate(df['GICS Sector']) if sector == 'Information Technology']
energy_indices = [i for i, sector in enumerate(df['GICS Sector']) if sector == 'Energy']
consumer_disc_indices = [i for i, sector in enumerate(df['GICS Sector']) if sector == 'Consumer Discretionary']
consumer_staples_indices = [i for i, sector in enumerate(df['GICS Sector']) if sector == 'Consumer Staples']
ny_indices = [i for i, location in enumerate(df['Location of Headquarters']) if 'New York, NY' in str(location)]

# Set objective: Maximize expected return
model.setObjective(gp.quicksum(df.iloc[i]['PercentReturn']/100 * x[i] for i in range(len(df))), GRB.MAXIMIZE)

# Constraint: Total investment of $10 million
model.addConstr(gp.quicksum(x[i] for i in range(len(df))) == 10000000, "Budget")

# Constraint: No more than $500,000 in Telecommunications sector
model.addConstr(gp.quicksum(x[i] for i in telecom_indices) <= 500000, "Telecom_Restriction")

# Constraint: IT investment at least 75% of Telecommunications investment
model.addConstr(gp.quicksum(x[i] for i in it_indices) >= 0.75 * gp.quicksum(x[i] for i in telecom_indices), "IT_Telecom_Restriction")

# Constraint: Absolute difference between Consumer Discretionary and Consumer Staples <= $200,000
model.addConstr(gp.quicksum(x[i] for i in consumer_disc_indices) - gp.quicksum(x[i] for i in consumer_staples_indices) <= 200000, "Disc_Staples_1")
model.addConstr(gp.quicksum(x[i] for i in consumer_staples_indices) - gp.quicksum(x[i] for i in consumer_disc_indices) <= 200000, "Disc_Staples_2")

# Constraint: At least $1 million in Energy sector
model.addConstr(gp.quicksum(x[i] for i in energy_indices) >= 1000000, "Energy_Sector")

# Constraint: At least $300,000 in companies headquartered in New York
model.addConstr(gp.quicksum(x[i] for i in ny_indices) >= 300000, "NYC_Investment")

# Solve the model
model.optimize()

# Print results
if model.status == GRB.OPTIMAL:
    print(f"Optimal Portfolio Return: ${model.objVal:.2f}")
    
    # Calculate investments by sector
    sector_investment = {}
    for sector in df['GICS Sector'].unique():
        indices = [i for i, s in enumerate(df['GICS Sector']) if s == sector]
        investment = sum(x[i].x for i in indices)
        sector_investment[sector] = investment
        print(f"Investment in {sector}: ${investment:.2f}")
    
    # Investment in NYC
    nyc_investment = sum(x[i].x for i in ny_indices)
    print(f"Investment in NYC companies: ${nyc_investment:.2f}")
    
    # Get shadow prices and other sensitivity info
    print("\nSensitivity Analysis:")
    print(f"Energy sector constraint shadow price: {model.getConstrByName('Energy_Sector').pi:.4f}")
    print(f"Telecom constraint shadow price: {model.getConstrByName('Telecom_Restriction').pi:.4f}")
    print(f"Budget constraint shadow price: {model.getConstrByName('Budget').pi:.4f}")
    
    # Find Coca-Cola in the dataset
    coca_cola_index = df[df['Company'].str.contains('Coca-Cola', na=False)].index
    if len(coca_cola_index) > 0:
        idx = coca_cola_index[0]
        print(f"\nCoca-Cola current return: {df.iloc[idx]['PercentReturn']:.2f}%")
        print(f"Coca-Cola investment: ${x[idx].x:.2f}")
        print(f"Coca-Cola RC: {x[idx].RC:.4f}")
else:
    print("No optimal solution found")

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 24.3.0 24D70)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 7 rows, 67 columns and 117 nonzeros
Model fingerprint: 0x5f89d287
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-03, 1e-01]
  Bounds range     [6e+05, 6e+05]
  RHS range        [2e+05, 1e+07]
Presolve removed 1 rows and 0 columns
Presolve time: 0.00s

Solved in 0 iterations and 0.00 seconds (0.00 work units)
Infeasible model
No optimal solution found
